In [1]:
import pandas as pd
import numpy as np
import math as m
import random as rand
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
from sklearn import linear_model as lm, metrics, ensemble as ens
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.svm import SVC
from sklearn.feature_selection import RFE, RFECV, SequentialFeatureSelector
import random

# Preprocessing - from George's code

In [2]:
#DEFINING A FUNCTION TO UPDATE COLUMN NAMES LATER
def lower_no_space(word): 
    
    word = re.sub(' ', '_', word) 
    
    word = re.sub(r'\'', '', word) 
    
    word = re.sub(r'\(', '', word)
    
    word = re.sub(r'\)', '', word)
    
    word = re.sub('\?', '', word)
    
    word = re.sub('/', '_', word)
    
    word = word.lower()
    
    return word

In [3]:
df_lean = pd.read_csv ('stats_by_sample.csv')

In [4]:
df_lean

,Unnamed: 0,sample_id,gene_1-Mar_rawscore,gene_1-Sep_rawscore,gene_11-Mar_rawscore,gene_2-Mar_rawscore,gene_2-Sep_rawscore,gene_4-Mar_rawscore,gene_4-Sep_rawscore,gene_5-Sep_rawscore,...,chromosome_2_mutations,chromosome_9_mutations,chromosome_16_mutations,chromosome_22_mutations,chromosome_17_mutations,chromosome_11_mutations,chromosome_12_mutations,chromosome_Y_mutations,chromosome_X_mutations,outcome
0,0,11__ungewickell__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,2,1,0,0,4,4,0,0,0,1
1,1,15__ungewickell__SS,0.000000,0.0,0.0,0.0,2.511819,0.0,0.0,0.0,...,4,0,2,1,2,5,0,0,1,1
2,2,16__ungewickell__MF,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,3,5,3,0,1,3,6,0,6,0
3,3,1__ungewickell__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,1,0,1,1,0,2,1,0,0,1
4,4,4__ungewickell__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0,0,1,0,3,2,3,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,134,WES_5__woollard__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,11,3,2,1,8,7,9,0,7,1
135,135,WES_6__woollard__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,10,7,5,1,5,10,4,0,3,1
136,136,WES_7__woollard__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,5,1,0,0,0,4,1,0,0,1
137,137,WES_8__woollard__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,5,1,0,0,0,3,2,0,1,1


In [5]:
df_clin = pd.read_csv("Homebase.csv", header = 1)

In [6]:
#RENAMING COLUMNS
df_clin = df_clin.rename(mapper = lower_no_space, axis = 1) 
df_clin.rename(columns={'subject_sample_id':'sample_id'}, inplace=True)

In [7]:
df_clin

,patient_id,data_access_group,sample_id,country_of_origin,country_of_residence,gender,race,ethnicity,date_of_initial_diagnosis,age_at_initial_diagnosis,...,date_of_relapse_disease_progression,date_of_relapse_disease_progression.1,date_of_relapse_disease_progression.2,date_of_relapse_disease_progression.3,disease_status_at_time_of_sampling,disease_status_at_time_of_data_entry,date_of_death,age_at_death,death_related_to_disease,complete
0,795-1,MD Anderson,spz-20,NaN,United States of America,Female,White,Not Hispanic or Latino or Spanish Origin,2005-05-05,72.0,...,2008-11-16,2010-11-26,2012-04-22,NaN,Progressive disease,Deceased,2013-03-10,80.0,Yes,Complete
1,795-2,MD Anderson,spz-3,NaN,United States of America,Female,White,Not Hispanic or Latino or Spanish Origin,2011-10-14,81.0,...,2012-06-22,2012-10-31,2013-03-06,2013-06-05,Progressive disease,Deceased,2015-08-19,85.0,Yes,Complete
2,795-3,MD Anderson,spz-4,NaN,United States of America,Male,White,Not Hispanic or Latino or Spanish Origin,2012-04-24,85.0,...,2013-05-14,2014-05-01,NaN,NaN,Stable disease,Deceased,2014-05-12,87.0,Yes,Complete
3,795-4,MD Anderson,spz-8,NaN,United States of America,Male,White,Not Hispanic or Latino or Spanish Origin,2010-07-09,65.0,...,2010-12-15,2012-02-24,NaN,NaN,Progressive disease,Deceased,2017-05-14,72.0,No,Complete
4,795-5,MD Anderson,spz-10,NaN,United States of America,Male,White,Not Hispanic or Latino or Spanish Origin,2012-06-25,87.0,...,2013-08-26,2014-02-24,NaN,NaN,Progressive disease,Deceased,2017-06-17,92.0,No,Complete
5,795-6,MD Anderson,spz-13,Germany,United States of America,Female,White,Not Hispanic or Latino or Spanish Origin,2012-12-12,76.0,...,2013-11-29,2015-11-03,2016-10-05,2017-01-03,Progressive disease,Deceased,2017-04-22,81.0,Yes,Complete
6,795-7,MD Anderson,spz-14,United States of America,United States of America,Male,White,Not Hispanic or Latino or Spanish Origin,2011-11-23,66.0,...,2013-05-15,NaN,NaN,NaN,Progressive disease,Deceased,2013-12-02,68.0,Yes,Complete
7,795-8,MD Anderson,spz-15,NaN,United States of America,Female,White,Not Hispanic or Latino or Spanish Origin,2012-04-22,69.0,...,2013-04-23,2014-04-03,2014-06-29,2015-04-26,Progressive disease,Deceased,2017-07-21,74.0,Yes,Complete
8,795-9,MD Anderson,spz-16,United States of America,United States of America,Female,American Indian or Alaska Native,Not Hispanic or Latino or Spanish Origin,2011-10-05,61.0,...,NaN,NaN,NaN,NaN,Progressive disease,Deceased,2014-10-05,64.0,Unknown,Complete
9,795-10,MD Anderson,spz-21,NaN,United States of America,Male,Black or African American,Not Hispanic or Latino or Spanish Origin,2012-07-26,62.0,...,2013-07-09,2013-11-26,NaN,NaN,Progressive disease,Deceased,2013-12-17,64.0,Yes,Complete


In [8]:
#TRANSFORM SAMPLE ID TO JOIN TO CLINICAL DATA
df_lean['sample_id'] = df_lean['sample_id'].apply(lambda x: re.sub('_', '-', x[:5]) if 'WES' in x else\
                                                  (x[:-10] if 'CTCL' in x else \
                                                   ('-'.join([ele.lstrip('0').lower() for ele in x[:-10].split('-')]) if 'SPZ' in x else x)))



In [9]:
#TONS OF DATA, PULL WHAT WE WANT
df_clin_lean = df_clin.drop(columns = [x for x in df_clin.columns if x not in ['gender', 'race', \
                                       'country_of_residence', 'sample_id', 'ethnicity',\
                                        'age_at_initial_diagnosis', 't', 'n', 'm', 'b',\
                                        'predominant_lesion_type_at_diagnosis','lymph_node_biopsy_performed',\
                                        'family_history_of_leukemia_lymphoma', \
                                        'has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical',\
                                        'cd4+:cd8+_ratio', 'total_lymphocyte_count', 'absolute_cd4+_count_per_ul',\
                                        '%cd4+cd26-', '%cd4+cd7-', 'tcr_clonality', 'tumor_cell_cd30+',\
                                        'large_cell_transformation', 'ldh_u_l', 'wbc_103_μl', 'rbc_106_μl',\
                                        'hematocrit_%', 'mcv_fl', 'mchc_g_dl', 'rdw_%', 'platelet_count_103_μl',\
                                        'segmented_neutrophil,_absolute_103_μl', 'lymphocyte,_absolute_103_μl',\
                                        'monocytes,_absolute_103_μl', 'eosinophils,_absolute_103_μl',\
                                        'basophils,_absolute_103_μl', 'segmented_neutrophils_%', 'lymphocytes_%',\
                                        'monocytes_%', 'eosinophils_%', 'basophils_%']])
                                       
                                       

In [10]:
# TURN YES/NO & POSITIVE/NEGATIVE TO DUMMIES
df_clin_lean['lymph_node_biopsy_performed'] = \
df_clin_lean['lymph_node_biopsy_performed'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_lean['family_history_of_leukemia_lymphoma'] = \
df_clin_lean['family_history_of_leukemia_lymphoma'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_lean['tumor_cell_cd30+'] = \
df_clin_lean['tumor_cell_cd30+'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_lean['large_cell_transformation'] = \
df_clin_lean['large_cell_transformation'].apply(lambda x: 1 if x == 'Yes' else 0)

df_clin_lean['tcr_clonality'] = \
df_clin_lean['tcr_clonality'].apply(lambda x: 1 if x == 'Positive' else 0)

df_clin_lean['has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical'] = \
df_clin_lean['has_the_patient_ever_been_exposed_at_work_or_in_the_service_to_a_toxic_chemical'].apply(lambda x: 1 if x == 'Yes' else 0)

In [11]:
#MERGE CLINICAL, GENETIC DATA
df_all = pd.merge(df_lean, df_clin_lean, on='sample_id', how='left')

In [12]:
df_all

,Unnamed: 0,sample_id,gene_1-Mar_rawscore,gene_1-Sep_rawscore,gene_11-Mar_rawscore,gene_2-Mar_rawscore,gene_2-Sep_rawscore,gene_4-Mar_rawscore,gene_4-Sep_rawscore,gene_5-Sep_rawscore,...,"segmented_neutrophil,_absolute_103_μl","lymphocyte,_absolute_103_μl","monocytes,_absolute_103_μl","eosinophils,_absolute_103_μl","basophils,_absolute_103_μl",segmented_neutrophils_%,lymphocytes_%,monocytes_%,eosinophils_%,basophils_%
0,0,11__ungewickell__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,15__ungewickell__SS,0.000000,0.0,0.0,0.0,2.511819,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,16__ungewickell__MF,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1__ungewickell__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4__ungewickell__SS,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,134,WES-5,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,8.1,16.5,0.3,3.8,0.3,NaN,NaN,NaN,NaN,NaN
135,135,WES-6,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,8.7,67.1,0.8,2.4,0.0,8.7,67.1,0.8,2.4,0.0
136,136,WES-7,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,1.9,28.7,0.9,0.0,0.0,NaN,NaN,NaN,NaN,NaN
137,137,WES-8,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,7.1,16.0,1.0,0.5,0.0,NaN,NaN,NaN,NaN,NaN


In [13]:
#IMPUTATION; "UNKNOWN" FOR CATEGORICAL, MEAN FILL-IN FOR CONTINUOUS
for col in df_clin_lean.columns:
    if col in ['race', 'gender', 'country_of_residence', 'ethnicity', 'predominant_lesion_type_at_diagnosis']:
        df_all[col] = df_all[col].fillna('unknown')
    elif col != 'sample_id':
        df_all[col] = df_all[col].fillna(np.mean(df_all[col]))

In [14]:
#GET DUMMIES FOR CATEGORICALS
df_all = pd.get_dummies(df_all, columns = ['race', 'gender', 'country_of_residence', 'ethnicity', 'predominant_lesion_type_at_diagnosis'])

#DEFINE STANDARDSCALER FOR LATER USE
std_scl = StandardScaler()

## Logistic Regression - From George

In [15]:
X = df_all.drop(columns = [x for x in df_all.columns if x == 'outcome' or x == 'sample_id'])
X_scaled = std_scl.fit_transform(X)
X_norm = normalize(X)

y = df_all.drop(columns = [x for x in df_all.columns if x != 'outcome'])

In [16]:
y.value_counts(normalize=True)

outcome
1          0.863309
0          0.136691
dtype: float64

In [17]:
# 10X ITERATED 3-FOLD CROSS-VALIDATED ACCURACY AND PRECISION FOR MOST ROBUST EVAL W/SMALL SAMPLE
rkf = RepeatedKFold(n_splits=3, n_repeats=10)

log = lm.LogisticRegression()
acc_scores = cross_val_score(log, X_scaled, y.values.ravel(), scoring='accuracy', cv=rkf, n_jobs=-1)
prec_scores = cross_val_score(log, X_scaled, y.values.ravel(), scoring='precision', cv=rkf, n_jobs=-1)
print('accuracy: ', np.mean(acc_scores))
print('precision: ', np.mean(prec_scores))

accuracy:  0.8632593277829173
precision:  0.8633672525439411


## Random Forest

In [18]:
# NOW RANDOM FOREST
rf = ens.RandomForestClassifier()
acc_scores = cross_val_score(rf, X, y.values.ravel(), scoring='accuracy', cv=rkf, n_jobs=-1)
prec_scores = cross_val_score(rf, X, y.values.ravel(), scoring='precision', cv=rkf, n_jobs=-1)
print('accuracy: ', np.mean(acc_scores))
print('precision: ', np.mean(prec_scores))

accuracy:  0.863259327782917
precision:  0.8638914585260563


## Ridge Regression

In [19]:
#RIDGE
for a in np.arange(1, 102, 10):
    rdg = lm.RidgeClassifier(alpha = a)
    acc_scores = cross_val_score(rdg, X_norm, y.values.ravel(), scoring='accuracy', cv=rkf, n_jobs=-1)
    prec_scores = cross_val_score(rdg, X_norm, y.values.ravel(), scoring='precision', cv=rkf, n_jobs=-1)
    print(a, ' ridge accuracy: ', np.mean(acc_scores))
    print(a, ' ridge precision: ', np.mean(prec_scores))
    


1  ridge accuracy:  0.8634289238359542
1  ridge precision:  0.8632901634289241
11  ridge accuracy:  0.8633209990749307
11  ridge precision:  0.8632130743139069
21  ridge accuracy:  0.8633209990749308
21  ridge precision:  0.8633672525439409
31  ridge accuracy:  0.8634597594819611
31  ridge precision:  0.8633826703669442
41  ridge accuracy:  0.8631822386679001
41  ridge precision:  0.863336416897934
51  ridge accuracy:  0.8633209990749308
51  ridge precision:  0.8631822386679002
61  ridge accuracy:  0.8632439099599138
61  ridge precision:  0.8631051495528832
71  ridge accuracy:  0.8633364168979341
71  ridge precision:  0.863166820844897
81  ridge accuracy:  0.863367252543941
81  ridge precision:  0.8632747456059204
91  ridge accuracy:  0.8634443416589577
91  ridge precision:  0.8633518347209374
101  ridge accuracy:  0.863259327782917
101  ridge precision:  0.8634443416589577


## SVM

In [20]:
#Support Vector Machine
for kern in ['linear', 'poly', 'rbf', 'sigmoid']:
    
    svc = SVC(kernel = kern, probability = True)
    
    acc_scores = cross_val_score(svc, X_norm, y.values.ravel(), scoring='accuracy', cv=rkf, n_jobs=-1)
    prec_scores = cross_val_score(svc, X_norm, y.values.ravel(), scoring='precision', cv=rkf, n_jobs=-1)
    print(kern, ' accuracy: ', np.mean(acc_scores))
    print(kern, ' precision: ', np.mean(prec_scores))

linear  accuracy:  0.8633980881899476
linear  precision:  0.8633055812519272
poly  accuracy:  0.8634289238359544
poly  precision:  0.8633518347209374
rbf  accuracy:  0.8634443416589578
rbf  precision:  0.8632747456059204
sigmoid  accuracy:  0.8632747456059205
sigmoid  precision:  0.8632747456059204


## XGBoost

In [21]:
#XG Boost
from numpy import loadtxt
from xgboost import XGBClassifier

model = XGBClassifier()
acc_scores = cross_val_score(model, X, y.values.ravel(), scoring='accuracy', cv=rkf, n_jobs=-1)
prec_scores = cross_val_score(model, X, y.values.ravel(), scoring='precision', cv=rkf, n_jobs=-1)
print('accuracy: ', np.mean(acc_scores))
print('precision: ', np.mean(prec_scores))

accuracy:  0.9079555966697501
precision:  0.9192214108668213


## ADABoost

In [22]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state=0)
acc_scores = cross_val_score(model, X, y.values.ravel(), scoring='accuracy', cv=rkf, n_jobs=-1)
prec_scores = cross_val_score(model, X, y.values.ravel(), scoring='precision', cv=rkf, n_jobs=-1)
print('accuracy: ', np.mean(acc_scores))
print('precision: ', np.mean(prec_scores))

accuracy:  0.9180388529139687
precision:  0.9389920692008807


## CATBoost

In [33]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations = 100, verbose=10,random_state=123)
acc_scores = cross_val_score(model, X, y.values.ravel(), scoring='accuracy', cv=rkf, n_jobs=-1)
prec_scores = cross_val_score(model, X, y.values.ravel(), scoring='precision', cv=rkf, n_jobs=-1)
print('accuracy: ', np.mean(acc_scores))
print('precision: ', np.mean(prec_scores))

/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight

Learning rate set to 0.13503
0:	learn: 0.5988834	total: 150ms	remaining: 2.85s
10:	learn: 0.2543702	total: 728ms	remaining: 596ms
19:	learn: 0.1594660	total: 1.14s	remaining: 0us
Learning rate set to 0.134409
0:	learn: 0.6149457	total: 41.7ms	remaining: 793ms
10:	learn: 0.2640693	total: 775ms	remaining: 634ms
19:	learn: 0.1110382	total: 1.41s	remaining: 0us
Learning rate set to 0.13503
0:	learn: 0.6091627	total: 58.9ms	remaining: 1.12s
10:	learn: 0.2058030	total: 896ms	remaining: 733ms
19:	learn: 0.1258843	total: 1.69s	remaining: 0us
Learning rate set to 0.030866
0:	learn: 0.6780611	total: 108ms	remaining: 10.7s
10:	learn: 0.5418467	total: 1.45s	remaining: 11.7s
20:	learn: 0.4415842	total: 2.51s	remaining: 9.45s
30:	learn: 0.3729739	total: 3.58s	remaining: 7.98s
40:	learn: 0.3185062	total: 4.73s	remaining: 6.81s
50:	learn: 0.2494352	total: 5.9s	remaining: 5.67s
60:	learn: 0.2030055	total: 7.07s	remaining: 4.52s
70:	learn: 0.1793930	total: 8.35s	remaining: 3.41s
80:	learn: 0.1586820	tot

/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight

Learning rate set to 0.13503
0:	learn: 0.6180702	total: 152ms	remaining: 2.89s
10:	learn: 0.2233785	total: 850ms	remaining: 696ms
19:	learn: 0.1536928	total: 1.47s	remaining: 0us
Learning rate set to 0.13503
0:	learn: 0.5985911	total: 54.8ms	remaining: 1.04s
10:	learn: 0.2227463	total: 765ms	remaining: 626ms
19:	learn: 0.1279829	total: 1.59s	remaining: 0us
Learning rate set to 0.030866
0:	learn: 0.6629260	total: 90ms	remaining: 8.91s
10:	learn: 0.4310446	total: 1.19s	remaining: 9.63s
20:	learn: 0.3126228	total: 2.27s	remaining: 8.55s
30:	learn: 0.2325708	total: 3.36s	remaining: 7.48s
40:	learn: 0.1858091	total: 4.45s	remaining: 6.4s
50:	learn: 0.1503542	total: 5.45s	remaining: 5.23s
60:	learn: 0.1268279	total: 6.58s	remaining: 4.21s
70:	learn: 0.1074648	total: 7.47s	remaining: 3.05s
80:	learn: 0.0925303	total: 8.54s	remaining: 2s
90:	learn: 0.0832020	total: 9.66s	remaining: 956ms
99:	learn: 0.0749123	total: 10.6s	remaining: 0us
Learning rate set to 0.030866
0:	learn: 0.6714142	total: 1

/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight

...

/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight

Learning rate set to 0.030724
0:	learn: 0.6673834	total: 198ms	remaining: 19.6s
10:	learn: 0.5146081	total: 1.19s	remaining: 9.66s
20:	learn: 0.4192829	total: 2.32s	remaining: 8.73s
30:	learn: 0.3445274	total: 3.54s	remaining: 7.87s
40:	learn: 0.2806984	total: 4.53s	remaining: 6.52s
50:	learn: 0.2192617	total: 5.52s	remaining: 5.3s
60:	learn: 0.1786842	total: 6.56s	remaining: 4.2s
70:	learn: 0.1519083	total: 7.45s	remaining: 3.04s
80:	learn: 0.1316237	total: 8.46s	remaining: 1.98s
90:	learn: 0.1195767	total: 9.47s	remaining: 937ms
99:	learn: 0.1081725	total: 10.4s	remaining: 0us
Learning rate set to 0.030866
0:	learn: 0.6673259	total: 82.3ms	remaining: 8.15s
10:	learn: 0.4738490	total: 1.33s	remaining: 10.8s
20:	learn: 0.3630388	total: 2.54s	remaining: 9.55s
30:	learn: 0.2910557	total: 3.76s	remaining: 8.37s
40:	learn: 0.2368655	total: 5s	remaining: 7.2s
50:	learn: 0.1969877	total: 6.25s	remaining: 6.01s
60:	learn: 0.1634519	total: 7.59s	remaining: 4.85s
70:	learn: 0.1417410	total: 8.8

/Users/adrian/Documents/Columbia/3_Fall_2022/Capstone/capstone/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Learning rate set to 0.030724
0:	learn: 0.6649936	total: 214ms	remaining: 21.2s
10:	learn: 0.4974558	total: 1.24s	remaining: 10s
20:	learn: 0.3732671	total: 2.47s	remaining: 9.29s
30:	learn: 0.3049214	total: 3.76s	remaining: 8.38s
40:	learn: 0.2534075	total: 4.96s	remaining: 7.14s
50:	learn: 0.2084271	total: 6.23s	remaining: 5.99s
60:	learn: 0.1722187	total: 7.48s	remaining: 4.78s
70:	learn: 0.1543513	total: 8.74s	remaining: 3.57s
80:	learn: 0.1349883	total: 10.1s	remaining: 2.37s
90:	learn: 0.1178918	total: 11.4s	remaining: 1.13s
99:	learn: 0.1025438	total: 12.2s	remaining: 0us
Learning rate set to 0.030866
0:	learn: 0.6707664	total: 114ms	remaining: 11.3s
10:	learn: 0.5111406	total: 1.19s	remaining: 9.65s
20:	learn: 0.4139194	total: 2.39s	remaining: 9s
30:	learn: 0.3485396	total: 3.5s	remaining: 7.79s
40:	learn: 0.2763443	total: 4.67s	remaining: 6.73s
50:	learn: 0.2320516	total: 5.78s	remaining: 5.55s
60:	learn: 0.1986947	total: 6.89s	remaining: 4.4s
70:	learn: 0.1731724	total: 8.02s

Learning rate set to 0.030866
0:	learn: 0.6707438	total: 166ms	remaining: 16.5s
10:	learn: 0.5156561	total: 1.19s	remaining: 9.64s
20:	learn: 0.4270154	total: 2.15s	remaining: 8.09s
30:	learn: 0.3532391	total: 3.19s	remaining: 7.09s
40:	learn: 0.2790129	total: 4.48s	remaining: 6.45s
50:	learn: 0.2277085	total: 5.48s	remaining: 5.26s
60:	learn: 0.1998892	total: 6.57s	remaining: 4.2s
70:	learn: 0.1802265	total: 7.67s	remaining: 3.13s
80:	learn: 0.1603572	total: 8.8s	remaining: 2.06s
90:	learn: 0.1445527	total: 9.94s	remaining: 983ms
99:	learn: 0.1303501	total: 10.9s	remaining: 0us
Learning rate set to 0.030866
0:	learn: 0.6721189	total: 72.1ms	remaining: 7.13s
10:	learn: 0.4902431	total: 1.5s	remaining: 12.2s
20:	learn: 0.3960008	total: 2.77s	remaining: 10.4s
30:	learn: 0.3304479	total: 3.9s	remaining: 8.68s
40:	learn: 0.2771744	total: 5.21s	remaining: 7.5s
50:	learn: 0.2345695	total: 6.39s	remaining: 6.14s
60:	learn: 0.2004687	total: 7.49s	remaining: 4.79s
70:	learn: 0.1805712	total: 8.